In [ ]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Json"

open Plotly.NET

#load "Helpers.fs"
#load "Bestiary.fs"
#load "Compare.fs"
#load "Library.fs"
#load "Transform.fs"

open PathfinderAnalysis.Library
open PathfinderAnalysis.Bestiary
open PathfinderAnalysis.Compare
open PathfinderAnalysis.Transform
open PathfinderAnalysis.Helpers

In [ ]:
[ Chart.Line(
    x = pcLevels,
    y = List.map (highMartialAttack true) pcLevels,
    Name = "Martial Attack Proficiency",
    ShowMarkers = true
);
Chart.Line(
    x = pcLevels,
    y = List.map (casterAttack true false) pcLevels,
    Name = "Caster Attack Proficiency",
    ShowMarkers = true
)]
|> Chart.combine
|> Chart.withLineStyle(Shape = StyleParam.Shape.Hvh)
|> Chart.withXAxisStyle(TitleText = "Character Level", MinMax = (0, 21))
|> Chart.withYAxisStyle(TitleText = "Attack Bonus")
|> Chart.withTitle "Martial-Caster Attack Disparity"


In [ ]:
let spoutPoints = 
  transformedResultsByRollByLevel spout CreatureSave middleSave (casterDc true) bestiaryByLevel 2
  |> resultsByRollByLevelToXyz
  |> Seq.map savingThrowNormalize
  |> Seq.toArray

let arbalestPoints =
  transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
  |> resultsByRollByLevelToXyz
  |> Seq.toArray 

[
  Chart.Scatter3D(
    xyz = spoutPoints,
    mode = StyleParam.Mode.Markers,
    Name = "Spout Damage"
  );
  Chart.Scatter3D(
    xyz = arbalestPoints,
    mode = StyleParam.Mode.Markers,
    Name = "Arbalest Damage"
  )
  // Chart.Mesh3D(
  //   xyz = spoutPoints,
  //   ShowLegend = true,
  //   Name = "Spout Damage",
  //   Opacity = 1.0
  // );
  // Chart.Mesh3D(
  //   xyz = arbalestPoints,
  //   ShowLegend = true,
  //   Name = "Arbalest Damage",
  //   Opacity = 0.5
  // )
]
|> Chart.combine
|> Chart.withXAxisStyle("Character Level", Id = StyleParam.SubPlotId.Scene 1)
|> Chart.withYAxisStyle("Player Roll Luck", Id = StyleParam.SubPlotId.Scene 1)
|> Chart.withZAxisStyle("Average Damage")
|> Chart.withTitle "Spout Cantrip vs. Martial Arbalest Against PL+2 Creatures"
|> Chart.withSize (800.0, 800.0)

In [ ]:
let spoutPoints = 
  transformedResultsByRollByLevel spout CreatureSave middleSave (casterDc true) bestiaryByLevel 2
  |> resultsByRollByLevelToRollsAndAveragesByLevel
  |> savingThrowNormalizeRollsAndAveragesByLevel
  |> Seq.toArray

let arbalestPoints =
  transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
  |> resultsByRollByLevelToRollsAndAveragesByLevel
  |> Seq.toArray 

Array.zip spoutPoints arbalestPoints
  |> Array.map (fun (spouts, arbalests) -> 
    let (level, spoutXy) = spouts
    let arbalestXy = second arbalests
    let spoutAverage = rollAveragesToAverage spoutXy
    let arbalestAverage = rollAveragesToAverage arbalestXy

    [
      Chart.Line(
        xy = spoutXy,
        Name = sprintf "Spout %.1f" spoutAverage,
        ShowMarkers = true
      );
      Chart.Line(
        xy = arbalestXy,
        Name = sprintf "Arbalest %.1f" arbalestAverage,
        ShowMarkers = true

      );
    ]
    |> Chart.combine
    |> Chart.withLineStyle(Shape = StyleParam.Shape.Hvh)
    |> Chart.withXAxisStyle("Player Roll Luck")
    |> Chart.withYAxisStyle("Average Damage")
    |> Chart.withTitle (sprintf "Middle Save - Level %i - Spout Cantrip vs. Martial Arbalest Against Level %i Creatures" level (level+2))
    |> Chart.withSize (800.0, 800.0)
  )